# Indhent wikipedia artikler

Formålet med denne notebook er at indhente Wikipedia-artikler der hører ind under minimum én af følgende tolv kategorier:

1.	UDANNELSE
2.	SAMFUND
3.	VIDENSKAB
4.	NATUR (KLIMA & MILJØ)
5.	TEKNOLOGI
6.	KULTUR (MUSIK, TEATER, KUNST)
7.	HISTORIE 
8.	SUNDHED
9.	GEOGRAFI
10.	ØKONOMI
11.	SPORT
12.	RELIGION
13. POLITIK
14. ERHVERVSLIV

Wikipedias artikler er inddelt i forskellige kategorier, som kan findes her: 

https://da.wikipedia.org/wiki/Wikipedia:Kategorier. 

Systemet er hierarkisk opbygget i en kringlet træ-struktur, hvor kategorier såsom *Historie*, *Sundhed*, *Geografi* m. fl. udgør topniveau-kategorier (https://da.wikipedia.org/wiki/Kategori:Topniveau_for_emner), som har en hel del underkategorier, der videre har en del underkategorier osv. Disse forgreninger af underkategorier kan sagtens være beslægtede - f.eks. hører Wikipedia artiklen "Arbejdsindsats" både ind under kategorierne *Sundhed* samt *Erhvervsliv*, som videre hører ind under *Økonomi* og *Samfund*. 

Kategorisystemet muliggør, at man kan indhente de Wikipedia-artikler, der hører ind under ovenstående tolv selvvalgte *hovedkategorier*. Dette gøres ved at starte med url-adresser for de tolv hovedkategorier. For hver hovedkategori, fx *Uddannelse*: "https://da.wikipedia.org/wiki/Kategori:Uddannelse", downloades først de tilhørende Wikipedia-artikler - for *Uddannelse* er det bl.a.artikler såsom Almendannelse, Elev, og Studiekort. Efterfølgende forfølges hvert link til underkategori-siderne, hvor de tilsvarende artikler ligeledes downloades osv. Da en artikel kan høre til flere kategorier vil denne måde at scrape på føre til artikel-gengangere. Disse vil vi håndtere i næste notebook.



# Indhent wikipedia url'er

Vi gemmer først url-adresserne for de tolv hovedkategorier i en liste

In [1]:
url_list = [
    "https://da.wikipedia.org/wiki/Kategori:Uddannelse",
    "https://da.wikipedia.org/wiki/Kategori:Samfund",
    "https://da.wikipedia.org/wiki/Kategori:Videnskab",
    "https://da.wikipedia.org/wiki/Kategori:Natur",
    "https://da.wikipedia.org/wiki/Kategori:Teknologi",
    "https://da.wikipedia.org/wiki/Kategori:Kultur",
    "https://da.wikipedia.org/wiki/Kategori:Historie",
    "https://da.wikipedia.org/wiki/Kategori:Sundhed",
    "https://da.wikipedia.org/wiki/Kategori:Geografi",
    "https://da.wikipedia.org/wiki/Kategori:%C3%98konomi",
    "https://da.wikipedia.org/wiki/Kategori:Sport",
    "https://da.wikipedia.org/wiki/Kategori:Religion",
    "https://da.wikipedia.org/wiki/Kategori:Politik",
    "https://da.wikipedia.org/wiki/Kategori:Erhvervsliv"
]

Herefter laver vi en funktion der vha. bibliotekerne *requests* og *bs4* givet en urladresse for en kategori scraper de tilhørende urladresser for hhv. underkategorier og artikler.

In [2]:
from bs4 import BeautifulSoup
import requests

In [3]:
def get_urls(URL, type_):
    main_url = "https://da.wikipedia.org"
    response = requests.get(URL)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    url_list = []
    if type_ == "subcategories":
        
        try:
            data = soup.findAll('div',attrs={'id':'mw-' + type_})
            for tag in data[0].find_all("a"):
                url_list.append(main_url + tag.attrs["href"])
            return url_list
        except:
            return []
    elif type_ == "pages":
        try:
            data = soup.findAll('div',attrs={'id':'mw-' + type_})
            for tag in data[0].find_all("a"):
                page_name = tag.attrs["href"]
                if not "Bruger:" in page_name and not "Portal:" in page_name:
                  url_list.append(main_url + page_name)
            return url_list
        except:
            return []

In [4]:
# Test funktion
print("Kategori:", url_list[6])
print(get_urls(url_list[6], "pages")[-3:])

Kategori: https://da.wikipedia.org/wiki/Kategori:Historie
['https://da.wikipedia.org/wiki/Liste_over_turistattraktioner_med_reenactment', 'https://da.wikipedia.org/wiki/Universalhistorie', 'https://da.wikipedia.org/wiki/Vestlig_historisk_t%C3%B8jmode']


For at gemme url-adresserne for hver hovedkategori benytter vi en dictionary med hovedkategorierne som keys. For hver key er den respektive value en liste med de tilhørende Wikipedia artikler. 

In [5]:
# Dictionary 
d = {"Uddannelse" : [],
    "Samfund": [],
    "Videnskab": [],
    "Natur": [],
    "Teknologi": [],
    "Kultur": [],
    "Historie": [],
    "Sundhed": [],
    "Geografi": [],
    "Økonomi": [],
    "Sport": [],
    "Religion": [],
    "Politik": [],
    "Erhvervsliv": []}
CATEGORIES = list(d.keys())

I det følgende looper vi gennem de tolv hovedkategorier. For hver hovedkategorier finder vi underkategorier ned til en dybde af 2 og scraper de respektive url-adresser for Wikipedia artiklerne.

In [6]:
%%time
DEPTH = 2 # Hvor mange undersider der scrapes
count = 1
for URL,category in zip(url_list,CATEGORIES):
    PAGE_URLS = []
    SUB_CATEGORIES = [[URL]]

    # get page urls - 1st layer
    for layer in range(DEPTH):
        s = []
        for url in SUB_CATEGORIES[layer]:
            PAGE_URLS.extend(get_urls(url, "pages"))
            s.extend(get_urls(url, "subcategories"))
        SUB_CATEGORIES.append(s)
        
    # store urls for the category
    d[category] = PAGE_URLS

    print(f"{count}/{len(CATEGORIES)}")
    count+=1

1/14
2/14
3/14
4/14
5/14
6/14
7/14
8/14
9/14
10/14
11/14
12/14
13/14
14/14
CPU times: total: 1min 16s
Wall time: 3min 6s


In [7]:
# Check størrelsen af dictionary
for cat in CATEGORIES:
  print(cat + ":", len(d[cat]))

Uddannelse: 598
Samfund: 2717
Videnskab: 917
Natur: 662
Teknologi: 312
Kultur: 2114
Historie: 1011
Sundhed: 517
Geografi: 368
Økonomi: 1179
Sport: 533
Religion: 1282
Politik: 1142
Erhvervsliv: 588


Det ses at der er en skæv fordeling i hvor mange Wikipedia artikel url'er vi har hentet. Denne skævhed er vigtig at håndtere senere ifm. ML-modellen -- vi kigger på det til den tid.

# Gem kategori-dictionary med url'er

In [8]:
import json

In [9]:
with open("dictionary.json", "w") as f:
    json.dump(d, f)

# Indhent tekster for hver url

In [10]:
# Indlæs dictionary
with open("dictionary.json", "r") as f:
    dic = json.load(f) 

Følgende funktion indhenter artiklerne for alle de url'er der findes i vores hovedkategory dictionary. 

In [11]:
import re
import time
import numpy as np

In [12]:
def get_plain_text(URL, max_tokens=-1):
  # Try to request wikipedia text
  try:
    response = requests.get(URL)
    soup = BeautifulSoup(response.text, 'html.parser')
    s = soup.findAll('div',attrs={'id':"mw-content-text"})[0]
    content_text = s.findAll('div')[0]
  except:
    print(f"Failed for URL: {URL}. We try to obtain the wikitext again in 5 seconds...")
    try:
      time.sleep(5)
      response = requests.get(URL)
      soup = BeautifulSoup(response.text, 'html.parser')
      s = soup.findAll('div',attrs={'id':"mw-content-text"})[0]
      content_text = s.findAll('div')[0]
    except:
      print("Failed again - no text is collected.")
      return np.nan, np.nan

  # Get the text
  plain_text = ""
  for c in content_text:
      if not c.name in ["style", "div", "h1", "h2", "h3", "h4", "h5", "h6", "table", "dl"]:
          try:
            plain_text = plain_text + " " + c.text
          except:
            pass
  
  # Process the text
  plain_text = " ".join(plain_text.split())
  plain_text = re.sub("[\(\[].*?[\)\]]", "", plain_text).replace("  ", " ")
  plain_text = plain_text.replace("\'", "").replace("\"", "").replace("”", "").replace(" .", ".").replace(" ,", ",")
  
  # Get title
  try:
      title = soup.title.text.replace("- Wikipedia, den frie encyklopædi","").strip()
  except:
      title = "Title not detected"

  # Return text + title
  return " ".join(plain_text.split()[:max_tokens]), title

# Test funktion
get_plain_text('https://da.wikipedia.org/wiki/Milit%C3%A6rf%C3%A6ngsel')

('Et militærfængsel er et fængsel drevet og styret af militæret. Militære fængsler bruges til at huse krigsfanger, ulovlige kombattanter, personer hvis frihed anses som en trussel mod den nationale sikkerhed, eller ansatte ved militæret fundet skyldig ved en krigsret i en alvorlig forbrydelse. United States Disciplinary Barracks',
 'Militærfængsel')

## Scrape Wikipedia artikler

In [13]:
%%time
text_list = []
label_num_list = []
label_list = []
title_list = []

for ix, cat in enumerate(CATEGORIES):
    for url in dic[cat]:
        [text, title] = get_plain_text(url)
        if len(text)>1:
            text_list.append(text)
            label_num_list.append(ix)
            label_list.append(cat)
            title_list.append(title)
    print(cat + f" {ix+1}/{len(CATEGORIES)}")



Uddannelse 1/14
Samfund 2/14
Videnskab 3/14
Natur 4/14
Teknologi 5/14
Kultur 6/14
Historie 7/14
Sundhed 8/14
Geografi 9/14
Økonomi 10/14
Sport 11/14
Religion 12/14
Politik 13/14
Erhvervsliv 14/14
CPU times: total: 17min 35s
Wall time: 1h 6min 13s


# Gem tekster i csv-fil

In [14]:
import pandas as pd

In [15]:
dict_df = {"text":text_list, "label_num": label_num_list, "label": label_list, "title_list":title_list}

In [16]:
df = pd.DataFrame.from_dict(dict_df)

In [17]:
df.head()

,text,label_num,label,title_list
0,"Uddannelse er et udtryk, der både benyttes om ...",0,Uddannelse,Uddannelse
1,30 er en dansk dokumentarfilm fra 2013 instrue...,0,Uddannelse,30 (dokumentarfilm)
2,"Accessit fra latin for han er kommet nær til, ...",0,Uddannelse,Accessit
3,"Almen dannelse er et begreb, der bruges om en ...",0,Uddannelse,Almendannelse
4,"En alumnus eller alumne er altid en person, me...",0,Uddannelse,Alumne


In [18]:
df.tail()

,text,label_num,label,title_list
13921,"En selvejende institution er et foretagende, d...",13,Erhvervsliv,Selvejende institution
13922,"En skibsreder er en person, der ejer et eller ...",13,Erhvervsliv,Skibsreder
13923,"En social virksomhed er en virksomhed, der er ...",13,Erhvervsliv,Social virksomhed
13924,En socialøkonomisk virksomhed er en virksomhed...,13,Erhvervsliv,Socialøkonomisk virksomhed
13925,"Et § 60-selskab er en virksomhed, der er opret...",13,Erhvervsliv,§ 60-selskab


In [19]:
df.to_csv("total_raw_wikidata.csv", index=False)